In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
# Load acronym intergrated data
acronym = pd.read_csv("Library Checkouts — Heading acronym list — metadata.csv")
acronym.head()

,Branch Code,Branch Heading,Heading,Heading Explanation,Item Type Code,Item Type Explanation
0,ANN,Annerley,Title,Title of Item,AD-FICTION,Adult Fiction
1,ASH,Ashgrove,Author,Author of Item,AD-MAGS,Adult Magazines
2,BNO,Banyo,Call Number,Call Number of Item,AD-PBK,Adult Paperback
3,BRR,BrackenRidge,Item id,Unique Item Identifier,BIOGRAPHY,Biography
4,BSQ,Brisbane Square Library,Item Type,Type of Item (see next column),BSQCDMUSIC,Brisbane Square CD Music


In [ ]:
# Extract Branch data as a table
# Difference compared with the Library location table
# 1. They all go with "Library"
# 2. Mt should come with no period (.)
# 3. BrackenRidge is separate
branches = acronym.iloc[:, [0, 1]].dropna()
for index, row_data in branches.iterrows():
  branch_name = str(row_data.iloc[1])
  if "BrackenRidge" in branch_name:
    branches.iloc[index, 1] = branch_name.replace("BrackenRidge", "Bracken Ridge")
  if "Mt." in branch_name:
    branches.iloc[index, 1] = branch_name.replace("Mt.", "Mt")
  if "library" not in str(row_data.iloc[1]).lower():
    branches.iloc[index, 1] = str(row_data.iloc[1]) + " Library"
branches.to_csv('Branches.csv', index=False)

In [ ]:
# Extract Heading data as metadata
headings = acronym.iloc[:, [2, 3]].dropna()
headings
headings.to_csv('Headings.csv', index=False)

In [ ]:
# Extract ItemType data as a table
itemTypes = acronym.iloc[:, [4, 5]].dropna()
itemTypes.to_csv('ItemTypes.csv', index=False)

In [ ]:
# Load libraries' location data
locations = pd.read_csv("libraries-information-location.csv")
locations.head(2)

,Venue,Latitude,Longitude,Website,Address,Open,Parking,Phone,Summary,Facilities,Meeting rooms,geolocation
0,Annerley Library,-27.509341,153.033310,https://www.brisbane.qld.gov.au/libraries-venues-and-facilities/libraries/library-locations/annerley-library,"450 Ipswich Rd, Annerley",Monday: 10am-5pm\nTuesday: 10am-5pm\nWednesday: closed\nThursday: closed\nFriday: 10am-5pm\nSaturday: 9am-4pm\nSunday: closed\nClosed on public holidays,On street;\nOff street,07 3403 1735,"The Annerley Library features a variety of facilities and services, as well as plenty of activities and events. Limited wheelchair access, phone the library for more information.","Use the website link for collections, facilities, opening hours, parking, location, upcoming events for this library; Limited wheelchair access, phone the library for more information.",No,"-27.509341, 153.03331"
1,Ashgrove Library,-27.443876,152.987086,https://www.brisbane.qld.gov.au/libraries-venues-and-facilities/libraries/library-locations/ashgrove-library,"87 Amarina Ave, Ashgrove",Monday: 10am-6pm\nTuesday: 10am-8pm\nWednesday: 10am-6pm\nThursday: 10am-8pm\nFriday: 10am-6pm\nSaturday: 9am-4pm\nSunday: closed\nClosed on public holidays,On street;\nOff street,07 3407 1940,"The Ashgrove Library features a variety of facilities and services, as well as plenty of activities and events. It is wheelchair accessible and has community meeting rooms available for hire, including access to a kitchenette.","Use the website link for collections, facilities, opening hours, meeting rooms, parking, location, upcoming events and phone number information for this library; wheelchair accessible.",Yes;\nHearing loop;\nWheelchair accessibility;\nWheelchair accessible toilets;\nUse the website link for meeting room details,"-27.443876, 152.987086"


In [ ]:
# For SQL practice, there are several columns that are not important to our report. So just drop them as their insights are not significant.
# Notice that, the Opening hours greatly vary among libraries. Hence, it is reasonable to drop this as a redundant column.
# Create a list of essential columns
columns_to_keep = ['Venue', 'Address', 'Parking', 'Facilities', 'Meeting rooms']
# Keep only the essential columns and drop others
locations = locations[columns_to_keep]
locations.head(2)

,Venue,Address,Parking,Facilities,Meeting rooms
0,Annerley Library,"450 Ipswich Rd, Annerley",On street;\nOff street,"Use the website link for collections, facilities, opening hours, parking, location, upcoming events for this library; Limited wheelchair access, phone the library for more information.",No
1,Ashgrove Library,"87 Amarina Ave, Ashgrove",On street;\nOff street,"Use the website link for collections, facilities, opening hours, meeting rooms, parking, location, upcoming events and phone number information for this library; wheelchair accessible.",Yes;\nHearing loop;\nWheelchair accessibility;\nWheelchair accessible toilets;\nUse the website link for meeting room details


In [ ]:
# Let's dive deeper into Open, Parking, Facilities, and Meeting rooms because they contain detailed information that need to be split and explored.
# Street Parking: to simplify, we can map as Street Parking, Shopping Centre and Not available
# Facilities: just keep the text after semi-colon
# Meeting rooms: only Yes or No are enough to figure out
# Define a function to map parking types
for index, row_data in locations.iterrows():
  parking = str(row_data.iloc[2])
  if "street" in parking.lower():
      locations.iloc[index, 2] = "Street parking"
  elif "Shopping Centre" in parking:
      locations.iloc[index, 2] = "Shopping Centre"
  else:
      locations.iloc[index, 2] = "Not available"

  facilities = str(row_data.iloc[3])
  locations.iloc[index, 3] = facilities.split(";")[-1]

  meeting_room = str(row_data.iloc[4])
  if "no" in meeting_room.lower():
    locations.iloc[index, 4] = "No"
  else:
    locations.iloc[index, 4] = "Yes"

In [ ]:
# Looks great now, let's save this as a table
locations.to_csv('Locations.csv', index=False)

In [ ]:
# It's time to investigate the fact table of library checkout in 2024
# Load checkout data
checkouts = pd.read_csv("library-checkouts-2024.csv")
checkouts.head(5)

/tmp/ipython-input-18-3727569031.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  checkouts = pd.read_csv("library-checkouts-2024.csv")


,Title,Author,Call Number,Item Id,Item Type,Status,Language,Age,Checkout Library,Date
0,The abolition of Britain : from Winston Churchill to Theresa May /,"Hitchens, Peter,",941.085 HIT,34000106965783,NONFICTION,CHECKEDOUT,NaN,ADULT,KEN,2024-12-09T23:57:56+10:00
1,The snakehead : an epic tale of the Chinatown underworld and the American dream /,"Keefe, Patrick Radden,",364.137 KEE,34000117610220,NONFICTION,CHECKEDOUT,NaN,ADULT,KEN,2024-12-09T23:57:49+10:00
2,What birdo is that? : a field guide to bird-people /,"Robin, Libby,",598.072 ROB,34000118196203,NONFICTION,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:46+10:00
3,Women making waves : trailblazing surfers in and out of the water /,"Einzig, Lara,",797.32 EIN,34000116161050,NONFICTION,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:40+10:00
4,The next big thing /,"Colley, James,",AD-PBK COL,34000119419448,AD-PBK,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:37+10:00


In [ ]:
# Interesting! At the first glance, the "/" at the end of items' title should be removed to improve readability.
checkouts['Title'] = checkouts['Title'].str.replace(r' /$', '', regex=True)

# According to the acronym list, the default language is English, equivalent to NaN. To make it clear and precise, let's replace it.
checkouts['Language'] = checkouts['Language'].fillna('ENGLISH')

# Finally, the Date column is formatted to comply with the SQL DATETIME format. UTC +10:00 indicates Brisbane Timezone, which can be ignored.
checkouts['Date'] = pd.to_datetime(checkouts['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')

checkouts.head(5)

,Title,Author,Call Number,Item Id,Item Type,Status,Language,Age,Checkout Library,Date
0,The abolition of Britain : from Winston Churchill to Theresa May,"Hitchens, Peter,",941.085 HIT,34000106965783,NONFICTION,CHECKEDOUT,ENGLISH,ADULT,KEN,2024-12-09 23:57:56
1,The snakehead : an epic tale of the Chinatown underworld and the American dream,"Keefe, Patrick Radden,",364.137 KEE,34000117610220,NONFICTION,CHECKEDOUT,ENGLISH,ADULT,KEN,2024-12-09 23:57:49
2,What birdo is that? : a field guide to bird-people,"Robin, Libby,",598.072 ROB,34000118196203,NONFICTION,CHECKEDOUT,ENGLISH,ADULT,IPY,2024-12-09 23:55:46
3,Women making waves : trailblazing surfers in and out of the water,"Einzig, Lara,",797.32 EIN,34000116161050,NONFICTION,CHECKEDOUT,ENGLISH,ADULT,IPY,2024-12-09 23:55:40
4,The next big thing,"Colley, James,",AD-PBK COL,34000119419448,AD-PBK,CHECKEDOUT,ENGLISH,ADULT,IPY,2024-12-09 23:55:37


In [ ]:
# Awesome! However, this could be not ideal for SQL management system. To reduce redundancy following 3rd Normal Form, I should split this into two table.
# The descriptive attributes of item details can be stored in "Item" table...
items = checkouts[["Item Id","Title", "Author", "Call Number", "Item Type", "Language", "Age"]].drop_duplicates()

# ...while the transactional records can be treated as the primary fact table called "Checkout". Also rearranging columns and adding index column are applied.
checkouts = checkouts.drop(columns=["Title", "Author", "Call Number", "Item Type", "Language", "Age"])
checkouts = checkouts.reset_index(drop=True)
checkouts["Checkout Id"] = checkouts.index + 1
checkouts = checkouts[["Checkout Id", "Item Id", "Status", "Checkout Library", "Date"]]


In [ ]:
# Now, save those cleaned dataframes as tables before importing to MS SQL Project
items.to_csv('Items.csv', index=False)
checkouts.to_csv('Checkouts.csv', index=False)